# Unreachable Code

In [ ]:
'''
with open('python_code_text.txt', 'r') as infile:
    # Read the contents of the file into memory
    data = infile.read()  

# Return a list of the lines, breaking at line boundaries.
lines_list = data.splitlines()
'''

In [54]:
def read_func(filename):
    '''
    return: the file as a list of lines
    parameter: file name with its path 
    Reads the text file that includes the code line by line.
    ''' 
    with open(filename,'r') as file:
        return [line.strip() for line in file]

In [55]:
lines_list = read_func('test.txt')

In [57]:
def functions_lines(lines_list):
    '''
    return: List of each line with function definition
    parameter: list of lines 
    ''' 
    return [index+1 for index,line in enumerate(lines_list)
            if len(line.split()) >= 1 
            if line.split()[0] == 'def']

In [58]:
def_lines = functions_lines(lines_list)
def_lines

[1, 8, 14, 18, 25, 31, 37]

# False condition or True condition

In [26]:
def get_next_function_line(index):
    '''
    Loop from that index line until the next function
    return: next function line
    parameter: index line
    '''
    for def_line in def_lines:
        if def_line > index:
            return def_line

In [27]:
def false_true_conditions(lines_list):
    for index_line,line in enumerate(lines_list):
        if 'if' in line:
            if 'False' in line:
                print(f'Unrechable code in line {index_line+2}, False condition\n')

            elif 'True' in line:
                detect_until = get_next_function_line(index_line+1)

                for line_else_index,lines_until_def in enumerate(lines_list[index_line:detect_until]):
                    if 'else' in lines_until_def:
                        print(f'Unrechable code in line {line_else_index+2}, if statement always true, therefore else condition is unreachable\n')

# After return statement

In [28]:
def after_return_statement(lines_list):
    for index,line in enumerate(lines_list):
        if 'return' in line:
            next_line = lines_list[index+1].split()
            if len(next_line) >= 1: # not empty line
                # if the next line is neither 'else' statement nor new function
                if 'else' in next_line or 'def' in next_line:
                    pass
                else:
                    print(f'Unreachable code after return statement in line {index+2} \n')

# Same condition in if and else statement

In [29]:
import re

In [30]:
def save_condition(line, statement):
    cond = re.split(':',line)[0]
    cond = re.split(statement, cond)[1].strip()
    return cond

In [31]:
def duplicated_condition(lines_list):
    for index,line in enumerate(lines_list):
        if 'if' in line:
            condition = save_condition(line, 'if')
        # if not empty line
        if len(line) >= 1 and 'elif' in line: 
                elif_condition = save_condition(line, 'elif')
                # Detect if else condition is the same as elif condition
                if elif_condition == condition:
                    print(f'Unreachable code at line {index+2},the elif-statement can never be true as the same condition in if-statement has already been handled \n')

In [32]:
def unreachable_code(lines_list):
    false_true_conditions(lines_list)
    after_return_statement(lines_list)
    duplicated_condition(lines_list)

In [33]:
unreachable_code(lines_list)

Unrechable code in line 5, if statement always true, therefore else condition is unreachable

Unrechable code in line 10, False condition

Unreachable code after return statement in line 16 

Unreachable code after return statement in line 21 

Unreachable code at line 29,the elif-statement can never be true as the same condition in if-statement has already been handled 



In [59]:
def find (par,file_name,what):
    with open(file_name) as f:
    # reading each line    
            for line in f:
   
        # reading each word        
                for word in line.split():

            # Handle what you need to handle. i.e.     
                    if word == par and what == "word":
                        return(word)
                    if word == par and what == "line":
                        return(line)

            #else: print("not found")

In [60]:
def Line_OfDivid():
    LineOfDivid =[]
    for i in range(len(lines_list)):
        for j in lines_list[i]:
            if j=="/":
                LineOfDivid.append(lines_list[i])
    return LineOfDivid
   

In [61]:
Line_OfDivid=Line_OfDivid()
Line_OfDivid

['print(x/y)', 'q = x/0', 'z = 1/p']

In [62]:
def AfterDiv(Line_OfDivid):
    list_num = []
    for i in range(len(Line_OfDivid)):
        num = Line_OfDivid[i].split("/")
        list_num.append(num[1])
    return list_num

In [63]:
List_num=AfterDiv(Line_OfDivid)

In [64]:
List_num

['y)', '0', 'p']

In [65]:
def DividByZero(List_num):
    for i in range(len(List_num)):
        if List_num[i].isnumeric() == True:
            if int(List_num[i]) == 0:
                 print(" division by zero  ")
                    #raise Exception(f'Error: mageic error of line {i} the error is " {data[i]} "') 
            
        else: 
            
            var_div = find(List_num[i], "de.txt", "line")
            num = var_div.split("=")[1]
            res = int(num)
            if res == 0: 
                print(" division by zero  ")
            

In [73]:
DividByZero(List_num)

FileNotFoundError: [Errno 2] No such file or directory: 'de.txt'

In [68]:
def magic2(list):
    operations = ['>', '<', '==','!=', '**','+', '-', '/','*', 'return']
    for i in range(len(list)):
             for j in range(len(lines_list[i])):
                if len(lines_list[i]) != j+1:
                    isNumber =  lines_list[i][j+1].isnumeric()
                    for operation in operations:
                        if (operation == lines_list[i][j] and isNumber) :
                            print (i)
                            print(f'Error: mageic error of line {i} the error is " {lines_list[i]} "')

In [70]:
magic2(lines_list)

18
Error: mageic error of line 18 the error is " if x>2: "
32
Error: mageic error of line 32 the error is " q = x/0 "
33
Error: mageic error of line 33 the error is " f = p+10 "
37
Error: mageic error of line 37 the error is " for i>7: "
